# Unsupervised Classification of AOP Surface Directional Reflectance data in GEE

Authors: Bridget Hass, John Musinsky

Modified from Qiusheng Wu's [Machine Learning with Earth Engine - Unsupervised Classification](https://github.com/giswqs/geemap/blob/master/examples/notebooks/31_unsupervised_classification.ipynb)

Source: https://developers.google.com/earth-engine/clustering

The `ee.Clusterer` package handles unsupervised classification (or clustering) in Earth Engine. More details about each Clusterer are available in the reference docs in the Code Editor.

In this tutorial, we will use a GEE clusterer to carry out unsupervised classification of SRER, using data from 2017 (collected in good weather conditions "green"), and compare it to data from 2018 (mixed weather conditions).

In [ ]:
import ee, geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

## Create an interactive map

In [ ]:
Map = geemap.Map()
Map

## Add SRER 2021 Surface Directional Reflectance

In [ ]:
# Specify center location of SRER
geo = ee.Geometry.Point([-110.83549, 31.91068])

# Read in SRER Surface Directional Reflectance (SDR) Image from 2021
SRER_SDR2021 = ee.Image("projects/neon/D14_SRER/L3/DP3-30006-001_D14_SRER_SDR_2021")

# Mask layers to only show values > 0 (this hides the no data values of -9999) 
SRER_SDR2021mask = SRER_SDR2021.updateMask(SRER_SDR2021.gte(0.0000))

# Set the visualization parameters so contrast is maximized, and set display to show RGB bands 
visParams = {'min':2,'max':20,'gamma':0.9,'bands':['band053','band035','band019']};

Map.addLayer(SRER_SDR2021mask, visParams, 'SRER 2021');

Map.setCenter(-110.83549, 31.91068, 11);

## Remove water vapor absorption bands

In [ ]:
bands = SRER_SDR2021.get('system:band_names').getInfo()
bad_bands = ['band' + str(a) for a in range(190,212)] + ['band' + str(a) for a in range(280,314)] + ['band' + str(a) for a in range(416,427)]
good_bands = [b for b in bands if b not in bad_bands]

In [ ]:
len(good_bands)

In [ ]:
SRER2021sub = SRER_SDR2021.select(good_bands)

## Look at the image properties

props = geemap.image_props(SRER_SDR2021)
props.getInfo()

## Make training dataset
Note: there are several ways you can create a region for generating the training dataset. If you don't define a region, it will use the image footprint by default. For this example we will not specify a region.

In [ ]:
# Make the training dataset.
training = SRER2021sub.sample(**{
#     'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)

## Train the clusterer

In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

## Classify the image

In [ ]:
# Cluster the input using the trained clusterer.
result = SRER2021sub.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')

## Label the clusters

In [ ]:
legend_keys = ['1', '2', '3', '4']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072']

# Reclassify the map
result = result.remap([0, 1, 2, 3], [1, 2, 3, 4])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Save the result to a tif file.

In [ ]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

## Optional Exercise:

On your own, run the clustering algorithm on SRER 2019 and 2018 to see if you get similar results. How might weather conditions affect the classification?